# VulnRadar
**VulnRadar**, a custom-built tool designed to process and visualize vulnerability data efficiently. VulnRadar allows users to filter and analyze CVEs based on critical parameters such as CVSS scores, affected ports, and severity levels. This approach enables targeted insights into vulnerabilities impacting the network.

## Project Objectives:
1. To build a tool that processes vulnerability data, focusing on CVE analysis, allowing security professionals to efficiently analyze, filter, and visualize vulnerabilities.
2. To offer a clear overview of host information, ports, and associated vulnerabilities in a structured, interactive dashboard.
3. To integrate data from Shodan for gathering host and vulnerability information, and then enrich this data with detailed CVE information.

## Tools Used:
1. **Shodan API:** To collect host details, including open ports and associated vulnerabilities.
2. **Python:** The core programming language for implementing the tool.
3. **Libraries:** Libraries such as requests for fetching data from APIs, pandas for managing and visualizing the data, and ipywidgets for interactive UI elements.
4. **IPython Widgets:** For creating a dynamic, interactive dashboard that allows filtering and displaying data based on user input.
5. **Data Visualization:** Displaying host details, ports, and CVE data in a user-friendly tabular format.

## Methodologies:
1. **Data Collection:** Using the Shodan API to gather information about hosts and open ports, including any vulnerabilities linked to each port.
2. **CVE Enrichment:** Extracting unique CVEs from the data and enriching them with detailed information like CVSS scores, severity, and proposed actions from external sources like the CVE DB.
3. **Filtering and Analysis:** Applying filters (CVSS score, port, and severity) to refine data and present it in an organized manner.
4. **Data Presentation:** Using ipywidgets to build an interactive dashboard where users can dynamically filter the data, drill down into vulnerabilities, and view detailed information per host.

## Outcomes:
1. **Interactive Dashboard:** The tool provides an interactive dashboard where users can view host details, port information, and associated CVEs. It enables users to filter vulnerabilities based on different parameters like CVSS score, port number, and severity.
2. **Enhanced Vulnerability Detection:** The enriched CVE data provides actionable insights into each vulnerability, assisting users in identifying critical security risks.
3. **User Customization:** Users can customize how they view the data using filter sliders and dropdowns for targeted analysis.

In [12]:
from shodan import Shodan
from IPython.display import display, HTML, clear_output
import requests
import pandas as pd
import ipywidgets as widgets

In [13]:
SHODAN_API_KEY = 'rl89iPZ0hf7oVDyjz7jCHf65qEVKwawm'

In [14]:
api = Shodan(SHODAN_API_KEY)

In [15]:
def get_shodan_data(ips):
    host_details = []
    for ip in ips:
        try:
            host_info = api.host(ip)
            ports = []
            
            for host in host_info.get('data', []):
                port_info = {
                    'port': host.get('port', 'Unknown'),
                    'transport': host.get('transport', 'Unknown'),
                    'product': host.get('product', 'Unknown'),
                    'version': host.get('version', 'Unknown'),
                    'cpe23': host.get('cpe23', 'Unknown'),
                    'vulns': list(host.get('vulns', {}).keys())
                }
                ports.append(port_info)
            
            host_detail = {
                'ip': host_info.get('ip_str'),
                'isp': host_info.get('isp', 'Unknown'),
                'org': host_info.get('org', 'Unknown'),
                'country': host_info.get('country_name', 'Unknown'),
                'city': host_info.get('city', 'Unknown'),
                'os': host_info.get('os', 'Unknown'),
                'ports': ports
            }
            host_details.append(host_detail)
        except Shodan.APIError as e:
            print(f"Error retrieving data for {ip}: {e}")
    return host_details

In [16]:
def get_unique_cves(host_details):
    # Create a set to store unique CVE IDs
    cve_set = set()

    # Iterate through each host in the host_details list
    for host_detail in host_details:
        # Iterate through the ports of each host
        for port_info in host_detail.get('ports', []):
            # Add all CVEs for each port to the set (set ensures uniqueness)
            cve_set.update(port_info.get('vulns', []))
    
    # Convert the set of CVEs back to a list and return
    return list(cve_set)


In [17]:
def get_cves_data(cve_set):
    CVEDB_URL = "https://cvedb.shodan.io/cve/"
    cves_details = []
    
    for cve in cve_set:
        url = CVEDB_URL + cve
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()

            cve_info = {
                'cve_id': cve,
                'summary': data.get('summary', 'No Summary'),
                'cvss': data.get('cvss', 'Unknown'),
                'epss': data.get('epss', 'Unknown'),
                'propose_action': data.get('propose_action', 'Unknown')
            }
            cves_details.append(cve_info)
        else:
            print(f"Error Fetching {cve}: {response.status_code}")

    return cves_details

In [18]:
def get_severity(cvss_score):
    """
    Determine severity based on CVSS score.
    """
    if cvss_score >= 9.0:
        return "Critical"
    elif 7.0 <= cvss_score < 9.0:
        return "High"
    elif 4.0 <= cvss_score < 7.0:
        return "Medium"
    elif 0.1 <= cvss_score < 4.0:
        return "Low"
    else:
        return "Unknown"  # For invalid or missing scores

In [19]:
def extract_vulnerable_ports(host_details):
    vulnerable_ports = []
    
    for host in host_details:
        ip = host['ip']
        for port in host['ports']:
            vulns = port.get('vulns', [])
            for cve in vulns:
                vulnerable_ports.append({
                    'ip': ip,
                    'port': port['port'],
                    'cve': cve
                })
    
    return vulnerable_ports

In [20]:
def enrich_vulnerabilities_with_cve_data(vulnerable_ports):
    # Extract unique CVEs
    cve_set = {item['cve'] for item in vulnerable_ports}
    cve_details = get_cves_data(cve_set)
    
    # Create a mapping of CVE ID to its details
    cve_map = {cve['cve_id']: cve for cve in cve_details}
    
    # Enrich the vulnerable_ports with CVE details
    enriched_data = []
    for entry in vulnerable_ports:
        cve_info = cve_map.get(entry['cve'], {})
        cvss_score = cve_info.get('cvss', 'Unknown')
        # Convert CVSS to a float if it's valid; otherwise, keep as "Unknown"
        try:
            cvss_score = float(cvss_score) if cvss_score != 'Unknown' else 'Unknown'
        except ValueError:
            cvss_score = 'Unknown'
        
        enriched_entry = {
            'ip': entry['ip'],
            'port': entry['port'],
            'cve_id': entry['cve'],
            'summary': cve_info.get('summary', 'No Summary'),
            'cvss': cvss_score,
            'severity': get_severity(cvss_score) if cvss_score != 'Unknown' else 'Unknown',
            'epss': cve_info.get('epss', 'Unknown'),
            'propose_action': cve_info.get('propose_action', 'Unknown')
        }
        enriched_data.append(enriched_entry)
    
    return enriched_data

In [21]:
def display_host_details_with_cve(host_details, enriched_data):
    # Create the main container
    container = widgets.Output()

    # Display initial IP and port details
    def show_ip_details():
        clear_output(wait=True)
        with container:
            for host in host_details:
                display_ip_details(host)

    # Display details for a specific IP
    def display_ip_details(host):
        # Display IP information
        ip_html = f"""
        <div>
            <button onclick="toggleDetails('{host['ip']}')" 
                    style="padding: 5px 10px; background-color: #28a745; color: white; border: none; border-radius: 4px; cursor: pointer; margin-bottom: 5px;">
                {host['ip']}
            </button>
            <div id="{host['ip']}" style="display: none; margin-top: 10px;">
                <b>ISP:</b> {host['isp']}<br>
                <b>Organization:</b> {host['org']}<br>
                <b>Country:</b> {host['country']}<br>
                <b>City:</b> {host['city']}<br>
                <b>OS:</b> {host['os']}<br>
            </div>
        </div>
        """
        display(HTML(ip_html))

        # Display port table
        display_ports_table(host["ports"])

        # Display CVE dashboard
        display_cve_table(host["ip"])

        # Add JavaScript for toggling IP details visibility
        display(HTML("""
        <script type="text/Javascript">
        function toggleDetails(ip) {
            var detailsDiv = document.getElementById(ip);
            if (detailsDiv.style.display === "none") {
                detailsDiv.style.display = "block";
            } else {
                detailsDiv.style.display = "none";
            }
        }
        </script>
        """))

    # Generate ports table
    def display_ports_table(ports):
        rows = []
        for port in ports:
            rows.append(f"""
            <tr>
                <td style="width: 30px; text-align: center; word-wrap: break-word;">{port['port']}</td>
                <td style="width: 30px; text-align: center; word-wrap: break-word;">{port['transport']}</td>
                <td style="width: 150px; text-align: center; word-wrap: break-word;">{port['product']}</td>
                <td style="width: 150px; text-align: center; word-wrap: break-word;">{port['version']}</td>
                <td style="width: 250px; text-align: center; word-wrap: break-word;">{port['cpe23']}</td>
            </tr>
            """)

        table_html = f"""
        <table style="width: 100%; border-collapse: collapse; margin-top: 10px; border: 1px solid #ddd;">
            <thead style="background-color: #343a40; color: white;">
                <tr>
                    <th style="padding: 4px; text-align: center; border: 1px solid #ddd;">Port</th>
                    <th style="padding: 4px; text-align: center; border: 1px solid #ddd;">Transport</th>
                    <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">Product</th>
                    <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">Version</th>
                    <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">CPE</th>
                </tr>
            </thead>
            <tbody>
                {''.join(rows)}
            </tbody>
        </table>
        """
        display(HTML(table_html))

    # Generate the CVE dashboard for a specific IP
    def display_cve_table(host_ip):
        # Filter enriched data for the specific IP
        ip_cves = [entry for entry in enriched_data if entry["ip"] == host_ip]
        if not ip_cves:
            display(HTML("<p>No vulnerabilities found for this IP.</p>"))
            return

        # Create CVE DataFrame
        cve_df = pd.DataFrame(ip_cves)

        # Output area for filtered CVEs
        cve_table_output = widgets.Output()

        # CVSS filter slider
        cvss_slider = widgets.FloatSlider(
            value=0.0,
            min=0.0,
            max=10.0,
            step=0.1,
            description="Min CVSS:"
        )

        # Port filter dropdown
        port_options = sorted(set(cve_df["port"].astype(str)))  # Get unique ports as options
        port_filter = widgets.Dropdown(
            options=["All Ports"] + port_options,  # Add option to show all ports
            description="Filter by Port:",
            value="All Ports"
        )

        # Severity filter dropdown
        severity_filter = widgets.Dropdown(
            options=["All Severities", "Low", "Medium", "High", "Critical"],
            description="Filter by Severity:",
            value="All Severities"
        )

        # Filter CVEs based on slider, port, and severity
        def filter_cves(change):
            min_cvss = cvss_slider.value
            selected_port = port_filter.value
            selected_severity = severity_filter.value

            # Apply filters
            filtered_cve_df = cve_df[
                (cve_df["cvss"].astype(float) >= min_cvss) &
                ((selected_port == "All Ports") | (cve_df["port"].astype(str) == selected_port)) &
                ((selected_severity == "All Severities") | (cve_df["severity"] == selected_severity))
            ]

            renamed_cve_df = filtered_cve_df.rename(
                columns={
                    "port": "Port",
                    "cve_id": "CVE ID",
                    "summary": "Description",
                    "cvss": "CVSS",
                    "severity": "Severity",
                    "epss": "EPSS",
                    "propose_action": "Recommended Action"   
                }
            )
    
            with cve_table_output:
                cve_table_output.clear_output(wait=True)
                if not renamed_cve_df.empty:
                    # Build rows with HTML
                    rows = []
                    for _, row in renamed_cve_df.iterrows():
                        rows.append(f"""
                        <tr>
                            <td style="text-align: center; padding: 20px;">{row['Port']}</td>
                            <td style="text-align: center; padding: 20px;">{row['CVE ID']}</td>
                            <td style="text-align: justify; padding: 20px;">{row['Description']}</td>
                            <td style="text-align: center;">{row['CVSS']}</td>
                            <td style="text-align: center;">{row['Severity']}</td>
                            <td style="text-align: center;">{row['EPSS']}</td>
                            <td style="text-align: center; padding: 20px;">{row['Recommended Action']}</td>
                        </tr>
                        """)

                    # Build table HTML
                    table_html = f"""
                    <div style="max-height: 300px; overflow-y: auto; border: 1px solid #ddd; padding: 10px;">
                        <table style="width: 100%; border-collapse: collapse; border: 1px solid #ddd;">
                            <thead style="background-color: #343a40; color: white;">
                                <tr>
                                    <th style="width: 5%; padding: 4px; text-align: center; border: 1px solid #ddd;">Port</th>
                                    <th style="width: 15%; padding: 4px; text-align: center; border: 1px solid #ddd;">CVE ID</th>
                                    <th style="width: 35%; padding: 4px; text-align: center; border: 1px solid #ddd;">Description</th>
                                    <th style="width: 5%; padding: 4px; text-align: center; border: 1px solid #ddd;">CVSS</th>
                                    <th style="width: 10%; padding: 4px; text-align: center; border: 1px solid #ddd;">Severity</th>
                                    <th style="width: 10%; padding: 4px; text-align: center; border: 1px solid #ddd;">EPSS</th>
                                    <th style="width: 20%; padding: 4px; text-align: center; border: 1px solid #ddd;">Recommended Action</th>
                                </tr>
                            </thead>
                            <tbody>
                                {''.join(rows)}
                            </tbody>
                        </table>
                    </div>
                    """
                    display(HTML(table_html))
                else:
                    display(HTML("<p>No CVEs match the selected filter.</p>"))

        # Observe slider, port filter, and severity filter changes
        cvss_slider.observe(filter_cves, names="value")
        port_filter.observe(filter_cves, names="value")
        severity_filter.observe(filter_cves, names="value")

        # Initial CVE table display
        filter_cves(None)

        # Create three equal blocks and arrange them horizontally with padding
        filter_block_layout = widgets.Layout(width="33%", padding="10px")
        filters_hbox = widgets.HBox([
            widgets.VBox([cvss_slider], layout=filter_block_layout),
            widgets.VBox([port_filter], layout=filter_block_layout),
            widgets.VBox([severity_filter], layout=filter_block_layout),
        ])

        display(widgets.VBox([filters_hbox, cve_table_output]))

    # Initial display
    show_ip_details()
    display(container)


In [22]:
ips_to_scan = ['8.8.8.8', '122.17.145.180','12.175.6.47']
host_details = get_shodan_data(ips_to_scan)
vulnerable_ports = extract_vulnerable_ports(host_details)
enriched_data = enrich_vulnerabilities_with_cve_data(vulnerable_ports)
display_host_details_with_cve(host_details, enriched_data)
#display_details(enriched_data)

Output()

If the output is **"Error displaying widget: model not found"**, please check browser console using F12, you may find error like "Unsatisfied version 2.3.2 from _JUPYTERLAB.CORE_OUTPUT of shared singleton module @lumino/widgets (required ^1.37.2)", you may proceed to solve it before you be able to see the saved output

**OR**

simply **RESTART** the kernel and **RUN** the code again.